<a href="https://colab.research.google.com/github/hariPrasadCoder/10-pointer-analysis/blob/main/Preparing_the_dataset_with_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn import preprocessing

In [75]:
df = pd.read_csv('scraped_data.csv')

In [78]:
price_trend = []
for i in df['ticker']:
  try:
    stock = yf.Ticker(i)
    df1 = stock.history(period='1y')
    df1.reset_index(inplace=True)

    df1['year'] = pd.DatetimeIndex(df1['Date']).year
    df1['month'] = pd.DatetimeIndex(df1['Date']).month

    df1 = df1[df1.year != 2022]
    df1 = df1[['month', 'Close']]
    df1 = df1.groupby(['month']).median()


    def trendline(index,data, order=1):
        coeffs = np.polyfit(index, list(data), order)
        slope = coeffs[-2]
        return float(slope)

    index=[1,2,3,4,5,6,7,8,9,10,11,12]
    resultant=trendline(index,df1['Close'])
    price_trend.append(resultant)
  except:
    price_trend.append('NA')

df['price_trend'] = price_trend

- BRK.B: No data found, symbol may be delisted


In [82]:
df = df[df.price_trend != 'NA']

In [95]:
x = df['price_trend'].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x = x.reshape(-1,1)
x_scaled = min_max_scaler.fit_transform(x)
df['price_trend'] = x_scaled * 100
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,ticker,company_name,pe_ratio,current_ratio,roce,debt_to_equity_ratio,inventory_turnover_ratio,free_cash_flow,reserves_trend,quick_ratio,return_on_equity_ratio,return_on_investment,price_trend
0,0,ABT,abbott_laboratories,17.43,1.71,0.181770,2.32,0.98,6740.00,1472.8,1.19,0.4245,0.2471,13.154162
1,1,ABBV,abbvie,34.92,1.83,0.088340,1.13,0.84,5724.00,599.3,1.42,0.2139,0.1643,13.362954
2,2,ABMD,abiomed,32.24,1.02,0.093213,9.96,1.42,16790.00,-738.8,0.91,0.5686,0.1881,13.998170
3,3,ACN,accenture,110.85,7.64,0.168375,0.11,0.51,221.20,221.2,6.89,0.1128,inf,22.183968
4,4,AAP,advance_auto_parts,44.55,2.24,0.209167,0.56,1.17,777.00,707.6,1.71,0.5988,0.4384,17.394117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,317,XLNX,xilinx,23.18,0.78,0.042560,2.79,2.23,-2521.00,497.2,0.68,0.1060,0.0615,18.107216
318,318,XYL,xylem,66.05,6.81,0.153830,0.77,1.06,1043.56,-77.2,6.35,0.2666,0.1852,14.393375
319,319,YUM,yum!_brands,47.03,2.33,0.054018,1.63,1.17,641.00,243.3,1.81,0.1531,0.1138,14.191796
320,320,ZBH,zimmer_biomet,37.96,0.96,0.184733,1.08,1.80,895.00,397.6,0.69,0.3394,0.2886,9.304188


In [96]:
df.to_csv('final_data.csv', index=False)
